In [1]:
# from collections import OrderedDict
# import json
# import torch
# import torch.nn as nn
# from mmcv.runner import load_checkpoint
# import torch.utils.checkpoint
# import yaml
# import os 

d:\Miniconda\envs\myenv\Lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
# class LayerNorm(nn.LayerNorm):
#     def forward(self, x: torch.Tensor):
#         ret = super().forward(x)
#         return ret
# class QuickGELU(nn.Module):
#     def forward(self, x:torch.Tensor):
#         return x * torch.sigmoid(1.702 * x )

        
    

In [ ]:
# class ResidualAttentionBlock(nn.Module):
#     def __init__(self, d_model:int, n_head = int, attn_mask: torch.Tensor = None):
#         super().__init__()
        
#         self.attn = nn.MultiheadAttention(d_model, n_head)
#         self.ln_1 = LayerNorm(d_model)
#         self.mlp = nn.Sequential(OrderedDict([
#             ('c_fc', nn.Linear(d_model, d_model * 4 )) #*4 là thiết kế chuẩn trong các mô hình Transformer
            
#         ]))

In [94]:
from transformers import CLIPProcessor, CLIPModel
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import time
import zipfile
from zipfile import ZipFile
import io
import argparse
import io
import os
import time
import zipfile
from zipfile import ZipFile

import numpy as np
import torch
import torch.distributed as dist
import torch.nn.functional as F
from PIL import Image
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from tqdm import tqdm

In [95]:
class D(Dataset):
    def __init__(self, vids, zip_prefix, transform=None, processor = None, args=None, max_video_frames = 256):
        self.transform = transform #Biến đổi ảnh
        self.processor = processor
        self.zip_prefix = zip_prefix #Đường dẫn đến file zip chứa ảnh
        self.vids = vids #Danh sách video cần xử lý
        self.args = args #Các tham số khác
        self.max_video_frames = max_video_frames

    def __len__(self):
        return len(self.vids) #Trả về số lượng video cần xử lý

    def __getitem__(self, index):
        # Đọc và xử lý từng frame của video
        vid = self.vids[index] #Lấy video thứ index
        zip_path = "%s/%s/%s.zip" % (self.zip_prefix, vid[-2:], vid) #Đường dẫn đến file zip chứa ảnh. Ví dụ: 123456.zip
        img_tensor = torch.zeros(self.max_video_frames, 3, 224, 224) #Khởi tạo tensor frames của video với kích thước 224x224x3
        video_mask = torch.zeros(self.max_video_frames).long()  #Khởi tạo mask cho video với kích thước max_video_frames để đánh dấu frame nào có ảnh hợp lệ

        try:
            with ZipFile(zip_path, 'r') as handler: #Mở file zip chứa ảnh
                img_name_list = handler.namelist() #Lấy danh sách tên ảnh trong file zip
                img_name_list = sorted(img_name_list) #Sắp xếp tên ảnh theo thứ tự tăng dần

                for i, img_name in enumerate(img_name_list): #Lặp qua từng ảnh trong danh sách
                    i_img_content = handler.read(img_name) #Đọc nội dung ảnh
                    i_img = Image.open(io.BytesIO(i_img_content)) #Mở file ảnh từ dữ liệu nhị phân (i_img_content). Trả về đối tượng Image
                    if self.processor:
                        i_img_tensor = self.processor(images=i_img, return_tensors="pt").pixel_values.squeeze()
                    elif self.transform:
                        i_img_tensor = self.transform(i_img)
                    else:
                        i_img_tensor = ToTensor()(i_img)
                    # i_img_tensor = self.transform(i_img) #Biến đổi ảnh thành tensor
                    # i_img_tensor = self.processor(images=i_img, return_tensors="pt").pixel_values.squeeze()
                    img_tensor[i, ...] = i_img_tensor #Lưu tensor ảnh vào img_tensor
                    video_mask[i] = 1
                    #Đánh dấu frame thứ i là ảnh hợp lệ
            return img_tensor, video_mask, vid 
        except FileNotFoundError as e:
            print(f"File not found: {zip_path}, error: {e}")
            print(f"Error processing item {index}: {str(e)}")
           # Trả về một tensor rỗng hoặc một giá trị đặc biệt để đánh dấu lỗi
            return torch.zeros(1), torch.zeros(1), "error"


In [96]:
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
def _convert_image_to_rgb(image):
    return image.convert("RGB")
def transform(n_px):
    return Compose([
        Resize(n_px, interpolation=BICUBIC), #Resize kích thước ảnh về n_px, sử dụng BICUBIC
        CenterCrop(n_px), #Cắt ảnh ở giữa với kích thước n_px
        _convert_image_to_rgb, #Chuyển ảnh sang định dạng RGB
        ToTensor(), #Chuyển ảnh sang tensor
        # Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)), # Chuẩn hoá, các số trên là mean,std của RGB
    ])
    

In [97]:
with open(r"D:\NCKH-NEU\data\meta\vids.txt", encoding="utf-8") as f: 
        vids = [x.strip() for x in f]
len(vids)

529

In [98]:
data = D(vids,r"D:\NCKH-NEU\data\jpg_zips", transform= transform(224))
dataloader = DataLoader(
    dataset= data,
    batch_size= 2,
    num_workers= 0,
    drop_last= False,
)


In [99]:
output_path = r"D:\NCKH-NEU\data\feat_zip_new\feats.zip"
device = torch.device("cuda")
torch.cuda.empty_cache()
model_path = "openai/clip-vit-base-patch32"



In [100]:
# model_path = r"C:\Users\ORLab\.cache1\model\openai\clip-vit-large-patch14"
model = CLIPModel.from_pretrained(model_path, torch_dtype=torch.float16)
processor = CLIPProcessor.from_pretrained(model_path)
model.gradient_checkpointing_enable()
model.to(device)
    

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [101]:
with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_STORED) as output_handler:
    vid_set = set()
    for k,batch in tqdm(enumerate(dataloader)):
        img = batch[0].to(device)
        video_mask = batch[1].to(device)
        vids = batch[2]
        
        with torch.no_grad():
            # Xử lý hình ảnh bằng mô hình CLIP
            inputs = processor(images=img.view(-1, 3, 224, 224), return_tensors="pt", padding=True).to(device)
            outputs = model.get_image_features(**inputs)

            # Reshape outputs để phù hợp với kích thước ban đầu của batch
            outputs = outputs.view(img.shape[0], img.shape[1], -1)

            # Áp dụng mask
            masked_outputs = outputs * video_mask.unsqueeze(-1)

        # Xử lý kết quả (ví dụ: lưu vào file, thêm vào danh sách, v.v.)
        for i, vid in enumerate(vids):
            if vid not in vid_set:
                vid_set.add(vid)
                # Ở đây bạn có thể thêm mã để xử lý hoặc lưu kết quả cho mỗi video
                # Ví dụ: lưu đặc trưng của video vào file
                # np.save(f"features/{vid}.npy", masked_outputs[i].cpu().numpy())
                embedding = masked_outputs[i].cpu().numpy().astype(np.float16)
                buffer = io.BytesIO()
                np.save(buffer, embedding)
                output_handler.writestr(f'{vid}', buffer.getvalue())

        # In ra một số thông tin để kiểm tra
        print(f"Batch {k}: Processed {len(vids)} videos. Total unique videos: {len(vid_set)}")
        torch.cuda.empty_cache()
print(f'Embedding saved to {output_path}')
  

1it [00:02,  2.84s/it]

Batch 0: Processed 2 videos. Total unique videos: 2


2it [00:05,  2.99s/it]

Batch 1: Processed 2 videos. Total unique videos: 4


3it [00:08,  2.95s/it]

...

Batch 262: Processed 2 videos. Total unique videos: 526


264it [11:52,  2.71s/it]

Batch 263: Processed 2 videos. Total unique videos: 528


265it [11:53,  2.69s/it]

Batch 264: Processed 1 videos. Total unique videos: 529
Embedding saved to D:\NCKH-NEU\data\feat_zip_new\feats.zip


In [109]:
import os
import zipfile

file_path = r"D:\NCKH-NEU\data\feat_zip_new\feats.zip"

# Check if the file exists
if os.path.exists(file_path):
    print("File found!")
    
#     # Open the zip file
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        # List all contents of the zip file
        all_files = zip_ref.namelist()
        if all_files:
            first_file = all_files[0]
            print(first_file)
            with zip_ref.open(first_file) as npy_file:
                np_data = np.load(npy_file)
                print(np_data)
            
            
        
#         # Example: Extract all files to a directory
#         zip_ref.extractall(r"D:\NCKH-NEU\data\extracted_files")
        
#         # Example: Extract and read a specific file inside the zip
#         with zip_ref.open('your_file.npy') as npy_file:
#             import numpy as np
#             np_data = np.load(npy_file)
#             print(np_data)
# else:
#     print("File not found!")


File found!
Q100009.npy
[[-0.1615 -0.257  -0.5483 ...  0.767  -0.3672 -0.2637]
 [-0.1613 -0.2568 -0.551  ...  0.769  -0.3684 -0.2668]
 [-0.163  -0.2588 -0.5483 ...  0.7676 -0.3677 -0.2659]
 ...
 [-0.     -0.     -0.     ...  0.     -0.     -0.    ]
 [-0.     -0.     -0.     ...  0.     -0.     -0.    ]
 [-0.     -0.     -0.     ...  0.     -0.     -0.    ]]
